In [2]:
from custom_dataset import custom_collate, Dataset_MutationList, Dataset_TopN_emb
from models import *

# LOAD DATA
# canonical_mut_embeddings_esm2 = np.load('../aa/canonical_mut_embeddings_esm2.npy')
canonical_ave_embeddings_esm2 = np.load('../aa/canonical_mut_average_embeddings_esm2.npy')
data_dir = '../labeled_data/'
labeled_data = os.listdir(data_dir)
for ni,i in sorted(zip(labeled_data,range(len(labeled_data)))):print(i,'\t',ni)
data = labeled_data[21]
print('\n',data)
data_df = pd.read_csv(data_dir+data)
nlabels = len(data_df['int_label'].unique())
device = 'cuda:1'

# Create dataset
# dataset = Dataset_MutationList(data_df, canonical_mut_embeddings_esm2,device)
dataset = Dataset_MutationList(data_df, canonical_ave_embeddings_esm2,device)

# Create DataLoader
# dataloader = DataLoader(dataset, batch_size=100, shuffle=False, collate_fn=custom_collate)

## TEST/TRAIN SPLIT
test_size = .2
random_state = 42
batch_size = 1
indices = list(range(len(dataset)))

train_indices, test_indices = train_test_split(
    indices, 
    test_size=test_size, 
    random_state=random_state
)

train_dataset = Subset(dataset, train_indices)
test_dataset = Subset(dataset, test_indices)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate)

43 	 BINARYdata_CANCER_TYPE_0MinMutations_1696MinCancerType.csv
28 	 BINARYdata_CANCER_TYPE_0MinMutations_169MinCancerType.csv
2 	 BINARYdata_CANCER_TYPE_3MinMutations_1696MinCancerType.csv
39 	 BINARYdata_CANCER_TYPE_3MinMutations_169MinCancerType.csv
14 	 BINARYdata_CANCER_TYPE_DETAILED_0MinMutations_1696MinCancerType.csv
31 	 BINARYdata_CANCER_TYPE_DETAILED_0MinMutations_169MinCancerType.csv
12 	 BINARYdata_CANCER_TYPE_DETAILED_3MinMutations_1696MinCancerType.csv
35 	 BINARYdata_CANCER_TYPE_DETAILED_3MinMutations_169MinCancerType.csv
47 	 canonical_ref_barcode_refIdxs_map.pkl
27 	 data_CANCER_TYPE_0MinMutations_1696MinCancerType.csv
29 	 data_CANCER_TYPE_0MinMutations_169MinCancerType.csv
52 	 data_CANCER_TYPE_3MinMutations_1696MinCancerType.csv
21 	 data_CANCER_TYPE_3MinMutations_169MinCancerType.csv
13 	 data_CANCER_TYPE_DETAILED_0MinMutations_1696MinCancerType.csv
33 	 data_CANCER_TYPE_DETAILED_0MinMutations_169MinCancerType.csv
3 	 data_CANCER_TYPE_DETAILED_3MinMutations_1696Min

In [3]:
dataset[0]

(tensor([[ 0.0708, -0.1004,  0.0965,  ...,  0.1381, -0.3298,  0.6028],
         [-0.1024,  0.2950,  0.1107,  ...,  0.1083, -0.1077, -0.1759],
         [ 0.0956, -0.0093,  0.0032,  ..., -0.3538, -0.6821, -0.0153],
         ...,
         [ 0.0572,  0.3157,  0.0624,  ...,  0.1077, -0.0975, -0.1868],
         [-0.2116, -0.0350,  0.3335,  ...,  0.1835, -0.1566,  0.2194],
         [-0.0313, -0.5568, -0.0538,  ...,  0.1537, -0.4635,  0.0801]],
        device='cuda:1'),
 tensor(34, device='cuda:1'))

In [4]:
# majority classifier
print(len(data_df['int_label'].unique()))
sorted(data_df['int_label'].value_counts(),reverse=True)[0]/len(data_df['int_label'])

53


0.16158075053780574

In [6]:
import torch.optim as optim
from tqdm import tqdm

class Config:
    n_layer: int = 3
    input_dim: int = 640
    dropout: float = 0.0
    bias: bool = False
    n_labels: int = 17
    pooling : str = 'mean'
    norm_fn: nn.Module = nn.LayerNorm
    position_embedding: bool = False
    max_len: int = 30

print('n labels:',nlabels)
config = Config()
config.n_labels = nlabels

model = Classifier(config)
model.to(device)

num_epochs = 100
learning_rate = 0.001

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    with tqdm(enumerate(train_loader), total=len(train_loader),desc='TRAINING') as pbar:
        for batch_idx, (data, target) in pbar:
            optimizer.zero_grad()
            output = model(data)
            # assert False
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            pbar.set_postfix({'Epoch':f'{epoch+1}/{num_epochs}, Loss: {loss.item():.4f}'})
            if batch_idx % 20000 == 0:
                print('')

        # Evaluation
        model.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            for data, target in tqdm(test_loader,desc='TESTING'):
                output = model(data)
                _, predicted = torch.max(output.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()

        accuracy = 100 * correct / total
        print(f'Test Accuracy: {accuracy:.2f}%, ({correct} of {total})')

n labels: 53


TRAINING:   0%|          | 18/90367 [00:00<17:06, 88.00it/s, Epoch=1/100, Loss: 11.4872] 

TRAINING:  22%|██▏       | 20024/90367 [02:15<08:20, 140.50it/s, Epoch=1/100, Loss: 20.4324] 

TRAINING:  44%|████▍     | 40024/90367 [04:39<05:59, 140.00it/s, Epoch=1/100, Loss: 6.3390]  

TRAINING:  66%|██████▋   | 60023/90367 [07:00<03:34, 141.58it/s, Epoch=1/100, Loss: 3.2201]  

...

Test Accuracy: 29.26%, (6610 of 22592)


TRAINING:   0%|          | 15/90367 [00:00<10:05, 149.16it/s, Epoch=5/100, Loss: 2.4299]

TRAINING:  22%|██▏       | 20027/90367 [02:00<07:43, 151.77it/s, Epoch=5/100, Loss: 0.2077] 

TRAINING:  41%|████      | 37064/90367 [03:51<05:32, 160.23it/s, Epoch=5/100, Loss: 4.0344] 


KeyboardInterrupt: 